In [32]:
import numpy as np
import math
import sys
from collections import defaultdict
from math import log
import time
try:
    # Python 2.7
    import urllib2 as ur
    orl2 = True
except:
    #Python 3.4
    import urllib.request as ur
    orl2 = False
from collections import Counter

In [33]:
def computeInitialProb(trainDataFile,numOfStates,randomized=False):
    trainFile=open(trainDataFile,"r")
    metaDataLine = trainFile.readline()
    headerLine = metaDataLine.split(" ")
    numSequences = int(headerLine[0])
    distinctObservations= int(headerLine[1])#Total Number of Distinct Observations
    empiricalDistr=Counter()
    for n in range(numSequences):
        line = trainFile.readline()#Reading Sequences 1 by 1
        line=line.rstrip("\n")
        l = line.split(" ")
        l=l[1:]
        lDistr=Counter(l)
        empiricalDistr+=lDistr
    totalSymbolsSeen=sum(empiricalDistr.values())
    initialProb=[]
    if randomized:
        for i in np.arange(numOfStates):
            initialProb.append((1.0*empiricalDistr[str(i%distinctObservations)])/totalSymbolsSeen)
    else:
        numOfStates=min(numOfStates,distinctObservations)
        for i in np.arange(numOfStates):
            initialProb.append((1.0*empiricalDistr[str(i)])/totalSymbolsSeen)
    #initialProb=[(x)/np.min(initialProb) for x in initialProb]
    return (numOfStates,distinctObservations,initialProb)

In [34]:
def createRandomMatrixA(numOfStates):
    matrixA=np.ndarray(shape=(numOfStates,numOfStates),dtype=float)
    prob=1.0/(numOfStates)
    matrixA.fill(prob)
    return matrixA
def createRandomMatrixB(numOfStates,distinctObservations):
    matrixB=np.ndarray(shape=(numOfStates,distinctObservations),dtype=float)
    prob=1.0/(distinctObservations)
    matrixB.fill(prob)
    return matrixB

In [35]:
def scaleVector(ndArrayToScale,valueToDivide):
    if valueToDivide==0:
        ndArrayToScale=0
    else:
        ndArrayToScale/=float(valueToDivide)
def computeAlpha(observations,a,aTranspose,b,bTranspose,pi,alphaDP,shouldScale=True):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    alphaDpScaleTime0=0
    if shouldScale==False:
        alphaDP[0]=pi*bTranspose[observations[0]]
    else:
        alphaDP[0]=pi*bTranspose[observations[0]]
        scaleVector(alphaDP[0],np.sum(alphaDP[0]))
    for t in np.arange(1,timePts):
        for i in np.arange(statesC):
            alphaDP[t][i]=b[i][observations[t]]*(np.sum(alphaDP[t-1]*aTranspose[i]))
        if shouldScale:#Always True except the case when we are using this method to compute Likelihood
            alphaDP[t][i]=b[i][observations[t]]*(np.sum(alphaDP[t-1]*aTranspose[i]))
            scaleVector(alphaDP[t],np.sum(alphaDP[t]))
def getObservationLikelihood(newObserv,timePt,a,aTranspose,b,bTranspose,alphaDP,shouldScale=True):
    statesC=a.shape[0]
    for i in np.arange(statesC):
        alphaDP[timePt][i]=b[i][newObserv]*(np.sum(alphaDP[timePt-1]*aTranspose[i]))
    if shouldScale:
        scaleVector(alphaDP[timePt],np.sum(alphaDP[timePt]))
    return np.sum(alphaDP[timePt])
def computeCacheableAlpha(observations,a,aTranspose,b,bTranspose,pi,shouldScale=True):
    alphaDP=np.zeros(shape=(len(observations),a.shape[0]))# Count_of_Observations*Count_of_Hidden_States
    computeAlpha(observations,a,aTranspose,b,bTranspose,pi,alphaDP,shouldScale)
    return alphaDP

In [36]:
def computeBeta(observations,a,b,bTranspose,pi,betaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    betaDP[timePts-1].fill(1)
    for t in np.arange(timePts-2,-1,-1):
        betaDpScaleTimeT=0
        for i in np.arange(statesC):
            betaDP[t][i]=np.sum(a[i]*bTranspose[observations[t+1]]*betaDP[t+1])
        betaDPtSum=np.sum(betaDP[t])
        if betaDPtSum==0:
            betaDpScaleTimeT=0
        else:
            betaDpScaleTimeT=1.0/betaDPtSum
        betaDP[t]*=betaDpScaleTimeT
    return betaDP

In [37]:
def computeGammaDP(diGammaDP):
    return np.sum(diGammaDP,axis=(2))
def computeDiGammaDP(alphaDP,betaDP,a,b,bTranspose,observations):
    observationsC=len(observations)
    statesC=alphaDP.shape[1]
    diGammaDP=np.zeros(shape=(observationsC-1,statesC,statesC),dtype=float)
    for i in np.arange(statesC):
        for t in np.arange(observationsC-1):
            diGammaDP[t][i]=alphaDP[t][i]*a[i]*bTranspose[observations[t+1]]*betaDP[t+1]
    diGammaDenom=np.sum(diGammaDP,axis=(1,2))#Sum(0),groupby(1,2)
    for t in np.arange(observationsC-1):
        if diGammaDenom[t]==0:
            diGammaDP[t]=0
        else:
            diGammaDP[t]/=diGammaDenom[t]
    return diGammaDP

In [38]:
def computeTransitionProbabilityA(diGammaDP,gammaDP):
    diGammaIJSumMatrix=np.sum(diGammaDP,axis=(0))
    gammaDPISumMatrix=np.sum(gammaDP,axis=(0))
    statesC=diGammaIJSumMatrix.shape[0]
    for i in np.arange(statesC):
        if gammaDPISumMatrix[i]==0:
            diGammaIJSumMatrix[i]=0
        else:
            diGammaIJSumMatrix[i]/=gammaDPISumMatrix[i]
    return diGammaIJSumMatrix

In [39]:
def computeObsrProbNum(gammaDPT,i,vk,observations):
    gammaDPi=gammaDPT[i]
    return np.sum(gammaDPi[np.where(observations==vk)])
def computeTransitionProbabilityB(gammaDP,observations,observationDict):
    observations=observations[:len(observations)-1]#Remember DiGammaDP is defined only from 0 to T-2
    statesC=gammaDP.shape[1]
    observationsC=len(observationDict)
    newlyComputedObsrProbB=np.zeros(shape=(statesC,observationsC),dtype=float)
    gammaDPISumMatrix=np.sum(gammaDP,axis=(0))
    gammaDPT=gammaDP.transpose()
    for i in np.arange(statesC):
        for vk in observationDict:
            if gammaDPISumMatrix[i]==0:
                newlyComputedObsrProbB[i][vk]=0
            else:
                newlyComputedObsrProbB[i][vk]=computeObsrProbNum(gammaDPT,i,vk,observations)/gammaDPISumMatrix[i]
    return newlyComputedObsrProbB

In [40]:
#Change Convergence Criteria to be more reasonable/Useful
def isConverged(count,convergenceIters):
    if count>=convergenceIters:
        return True
    return False
def Forward_Backward_EM_Algo(observations,A,B,pi,convergenceIters,observationDict):
    count=0
    updatedA=A
    updatedB=B
    while isConverged(count,convergenceIters)==False:
        #Expectation(E)-Step
        alphaDP=np.zeros(shape=(observations.shape[0],updatedA.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        betaDP=np.zeros(shape=(observations.shape[0],updatedA.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        updatedATranspose=updatedA.transpose()
        updatedBTranspose=updatedB.transpose()
        computeAlpha(observations,updatedA,updatedATranspose,updatedB,updatedBTranspose,pi,alphaDP)
        computeBeta(observations,updatedA,updatedB,updatedBTranspose,pi,betaDP)
        #print("AlphaDP ",alphaDP)
        diGammaDP=computeDiGammaDP(alphaDP,betaDP,updatedA,updatedB,updatedBTranspose,observations)
        gammaDP=computeGammaDP(diGammaDP)#[t][state]
        #Maximization(M)-Step
        newA=computeTransitionProbabilityA(diGammaDP,gammaDP)
        newB=computeTransitionProbabilityB(gammaDP,observations,observationDict)
        updatedA=newA
        updatedB=newB
        count=count+1
    return (updatedA,updatedB)

In [41]:
def trainHMM(trainDataFile,A,B,pi,convergenceIters,maxSequences=-1):
    trainFile=open(trainDataFile,"r")
    metaDataLine = trainFile.readline()
    headerLine = metaDataLine.split(" ")
    numSequences = int(headerLine[0])
    distinctObservations= int(headerLine[1])#Total Number of Distinct Observations
    observationDict=np.arange(distinctObservations)
    updatedA=np.NaN
    updatedB=np.NaN
    isAUpdated=False
    if(maxSequences==-1):
        usedSeqs=numSequences
    else:
        usedSeqs=min(maxSequences,numSequences)
    actuallyUsedSeqs=0
    for n in range(usedSeqs):
        line = trainFile.readline()#Reading Sequences 1 by 1
        if n%11!=0:
            continue
        line=line.rstrip("\n")
        l = line.split(" ")
        if(int(l[0])<=1):
            continue
        actuallyUsedSeqs+=1
        observations=np.array([int(i) for i in l[1:len(l)]])
        #print("Observations ",observations)
        learnedParams=Forward_Backward_EM_Algo(observations,A,B,pi,convergenceIters,observationDict)
        if isAUpdated==False:
            isAUpdated=True
            updatedA=learnedParams[0]
            updatedB=learnedParams[1]
        else:
            updatedA+=learnedParams[0]
            updatedB+=learnedParams[1]
    updatedA=updatedA/actuallyUsedSeqs
    updatedB=updatedB/actuallyUsedSeqs
    return (updatedA,updatedB)

In [42]:
def trainModel(fileLoc,maxNoOfStates,convergenceIters,maxSequences=-1,initialProbRandomized=False):
    start = time.time()
    initialProbs=computeInitialProb(fileLoc,maxNoOfStates,initialProbRandomized)
    print("Inital Probs ",initialProbs)
    end = time.time()
    print("Computed Initial Prob. in ", end - start ,"seconds")
    pi=initialProbs[2]
    numOfStates=initialProbs[0]
    distinctObservations=initialProbs[1]
    A=createRandomMatrixA(numOfStates)
    B=createRandomMatrixB(numOfStates,distinctObservations)
    trainedParams=trainHMM(fileLoc,A,B,pi,convergenceIters,maxSequences)
    trainedParams=trainedParams+(pi,)
    end=time.time()
    print("For ",maxSequences," Sequences : Total Training Time ",end-start," seconds")
    return trainedParams

In [44]:
np.seterr(all='raise')
#(A,B,pi)=trainModel('Data/1.spice.train.txt',25,10,6000,True)
#(A,B,pi)=trainModel('Data/2.spice.train.txt',15,8,6000,True)
#(A,B,pi)=trainModel('Data/3.spice.train.txt',15,8,6000,True)
#(A,B,pi)=trainModel('Data/4.spice.train.txt',33,8,5000,True)
#(A,B,pi)=trainModel('Data/5.spice.train.txt',35,7,5000,True)
#(A,B,pi)=trainModel('Data/6.spice.train.txt',35,7,2000,True)
#(A,B,pi)=trainModel('Data/7.spice.train.txt',30,6,10000,True)
#(A,B,pi)=trainModel('Data/8.spice.train.txt',35,7,2000,True)
#(A,B,pi)=trainModel('Data/9.spice.train.txt',20,7,4000,True)
#(A,B,pi)=trainModel('Data/10.spice.train.txt',25,7,8000,True)
#(A,B,pi)=trainModel('Data/11.spice.train.txt',50,10,2000,True)
#(A,B,pi)=trainModel('Data/12.spice.train.txt',7,8,12000,True)
#(A,B,pi)=trainModel('Data/13.spice.train.txt',30,7,2000,True)
#(A,B,pi)=trainModel('Data/14.spice.train.txt',10,5,16000,True)
(A,B,pi)=trainModel('Data/15.spice.train.txt',15,7,10000,True)

Inital Probs  (15, 32, [0.028825739217717115, 0.05182187430039092, 0.02906732730265181, 0.016718289103688524, 0.04888935905352578, 0.01725263668055019, 0.03155947322972764, 0.009900683187486622, 0.053717676522902894, 0.07394760343143647, 0.0006179931459836596, 0.018085156109693783, 0.007220088713506545, 0.0048598075659877435, 0.02721334786470083])
Computed Initial Prob. in  8.412896871566772 seconds
For  10000  Sequences : Total Training Time  1064.2700824737549  seconds


In [45]:
def getAverageLikelihood(trainDataFile,A,B,pi,usedSeqs):
    trainFile=open(trainDataFile,"r")
    metaDataLine = trainFile.readline()
    ATranspose=A.transpose()
    BTranspose=B.transpose()
    totalObsr=0.0
    obsrCount=0
    for n in range(usedSeqs):
        line = trainFile.readline()#Reading Sequences 1 by 1
        #if n%2!=0:
        #    continue
        line=line.rstrip("\n")
        l = line.split(" ")
        if(int(l[0])<=1):
            continue
        observations=np.array([int(i) for i in l[1:len(l)]])
        t=len(observations)
        alphaDP=computeCacheableAlpha(observations,A,ATranspose,B,BTranspose,pi,True)
        obsrLikelihood=getObservationLikelihood(observations[t-1],t-1,A,ATranspose,B,BTranspose,alphaDP,False)
        totalObsr+=obsrLikelihood
        obsrCount+=1
    return totalObsr/obsrCount

In [46]:
getAverageLikelihood('Data/15.spice.train.txt',A,B,pi,20000)

0.023381058871187372

In [41]:
def getHmmRank(prefix,A,ATranspose,B,BTranspose,pi,uniqueSymbols):
    prefix.append(0)
    observations=np.array(prefix)
    alphaDP=computeCacheableAlpha(observations,A,ATranspose,B,BTranspose,pi,True)
    obsrLikelihood=getObservationLikelihood(0,len(observations)-1,A,ATranspose,B,BTranspose,alphaDP,False)
    prefix.pop()
    likelihoods=[]
    for i in np.arange(0,uniqueSymbols):
        prefix.append(i)
        observations=np.array(prefix)
        obsrLikelihood=getObservationLikelihood(i,len(observations)-1,A,ATranspose,B,BTranspose,alphaDP,False)
        prefix.pop()
        likelihoods.append((i,obsrLikelihood))
    likelihoods=sorted(likelihoods, key=lambda x: -x[1])
    ranks=[i[0] for i in likelihoods]
    return ranks

In [42]:
def list_to_string(l):
    s=str(l[0])
    for x in l[1:]:
        s+= " " + str(x)
    return(s)
def formatString(string_in):
    """ Replace white spaces by %20 """
    return string_in.strip().replace(" ", "%20")
# get the test first prefix: the only element of the test set
def get_first_prefix(test_file):
    """ This function is called for the public test file(Which only has 1 line)
    """
    f = open(test_file)
    prefix = f.readline()
    f.close()
    return prefix
def predictOnSpicePublicData(problem_number,name):
    problem_number = str(problem_number)
    user_id = '68'
    #name = "hmm_Baseline"
    #train_file = 'Data/0.spice.train.txt'
    prefix_file = 'Data/'+problem_number+'.spice.private.test.txt'
    first_prefix = get_first_prefix(prefix_file)
    prefix_number=1
    # get the next symbol ranking on the first prefix
    p=first_prefix.split()
    prefix=[int(i) for i in p[1:len(p)]]#prefix holds the sequence of values in the public test file(Note:It has only 1 Seq)
    print("Prefix ",prefix)
    ranking=getHmmRank(prefix,A,A.transpose(),B,B.transpose(),pi,B.shape[1])
    print("Model Ranking ",ranking)
    ranking_string=list_to_string(ranking[:5])
    #print("Prefix number: " + str(prefix_number) + " Ranking: " + ranking_string + " Prefix: " + first_prefix)
    first_prefix = formatString(first_prefix)

    # transform the ranking to follow submission format
    ranking_string=formatString(ranking_string)

    # create the url to submit the ranking
    #name=name+"_Ver1.7.2"
    name=name
    url_base = 'http://spice.lif.univ-mrs.fr/submit.php?user=' + user_id +\
        '&problem=' + problem_number + '&submission=' + name + '&'
    url = url_base + 'prefix=' + first_prefix + '&prefix_number=1' + '&ranking=' +\
        ranking_string
    response = ur.urlopen(url)
    print("URL ",url)
    content = response.read()
    print("Response from SPiCe ",content)#Content is a new Sequence returned from the SPiCe server: We will need to predict for this seq
    if not orl2:
        # Needed for python 3.4...
        content= content.decode('utf-8')
    list_element = content.split()
    head = str(list_element[0])
    return content,url_base

In [43]:
spiceContentOnPubFile,url_base=predictOnSpicePublicData(15,"hmm_alok_v15.1")

Prefix  [16, 27, 28, 13, 16, 18, 27]
Model Ranking  [16, 9, 15, 29, 1, 28, 8, 4, 25, 27, 18, 23, 22, 6, 17, 2, 0, 14, 24, 11, 21, 5, 3, 19, 7, 20, 12, 30, 13, 26, 31, 10]
URL  http://spice.lif.univ-mrs.fr/submit.php?user=68&problem=15&submission=hmm_alok_v15.1&prefix=7%2016%2027%2028%2013%2016%2018%2027&prefix_number=1&ranking=16%209%2015%2029%201
Response from SPiCe  b'12 18 27 3 16 24 23 6 17 14 8 0 15\n'


In [44]:
def evaluateOnSpiceTrainDataSet(prevContent,url_base):
    prefix_number = 2
    head=''
    content=prevContent
    while(head != '[Error]' and head != '[Success]'):
        prefix = content[:-1]#Fetch the Sequence returned from Spice Server and exclude the last '\n'
        # Get the ranking
        p=prefix.split()
        prefix_list=[int(i) for i in p[1:len(p)]]
        ranking = getHmmRank(prefix_list,A,A.transpose(),B,B.transpose(),pi,B.shape[1])
        ranking_string=list_to_string(ranking[:5])#Here At least alphabet should be 4: Else may get Runtime error
        if prefix_number % 200 == 0:
            print("Prefix number: " + str(prefix_number) + " Ranking: " + ranking_string + " Prefix: " + prefix)
        # Format the ranking
        ranking_string = formatString(ranking_string)
        # create prefix with submission needed format
        prefix=formatString(prefix)
        # Create the url with your ranking to get the next prefix
        url = url_base + 'prefix=' + prefix + '&prefix_number=' +\
            str(prefix_number) + '&ranking=' + ranking_string
        # Get the answer of the submission on current prefix
        response = ur.urlopen(url)
        content = response.read()
        if not orl2:
            # Needed for Python 3.4...
            content= content.decode('utf-8')
        list_element = content.split()
        # modify head in case it is finished or an erro occured
        head = str(list_element[0])
        # change prefix number
        prefix_number += 1
    # Post-treatment
    # The score is the last element of content (in case of a public test set)
    print(content)
    list_element = content.split()
    score = (list_element[-1])
    print(score)

In [45]:
evaluateOnSpiceTrainDataSet(spiceContentOnPubFile,url_base)

Prefix number: 200 Ranking: 16 9 15 29 1 Prefix: 3 18 28 28
Prefix number: 400 Ranking: 16 9 15 29 1 Prefix: 15 18 27 23 16 23 29 28 8 17 3 28 15 27 29 4
Prefix number: 600 Ranking: 16 9 15 29 1 Prefix: 6 21 2 1 16 9 15
Prefix number: 800 Ranking: 16 9 15 29 1 Prefix: 14 6 1 15 2 15 25 1 6 3 19 29 4 0 28
Prefix number: 1000 Ranking: 16 9 15 29 1 Prefix: 7 18 27 29 23 28 8 15
Prefix number: 1200 Ranking: 16 9 15 29 1 Prefix: 22 12 12 13 22 4 1 15 25 4 15 9 27 9 20 27 16 28 8 11 17 8 22
Prefix number: 1400 Ranking: 16 9 15 29 1 Prefix: 35 4 4 18 15 28 19 29 4 0 2 16 29 14 8 15 21 4 25 9 16 14 1 9 16 25 28 25 4 1 25 8 22 9 18 18
Prefix number: 1600 Ranking: 16 9 15 29 1 Prefix: 4 16 29 14 19
Prefix number: 1800 Ranking: 16 9 15 29 1 Prefix: 1 6
Prefix number: 2000 Ranking: 16 9 15 29 1 Prefix: 26 6 1 15 16 28 26 23 29 1 20 24 21 18 15 2 14 16 18 27 16 24 27 3 29 9 18
Prefix number: 2200 Ranking: 16 9 15 29 1 Prefix: 4 16 18 27 16
Prefix number: 2400 Ranking: 16 9 15 29 1 Prefix: 68 16 29 

In [ ]:
evaluateOnSpiceTrainDataSet(spiceContentOnPubFile,url_base)